In [2]:
import pandas as pd
import re
import csv
from nltk import word_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import stem
from nltk import sent_tokenize
import sys

# Add the 'utils' directory to the system path
utils_path = '/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/utils'
sys.path.insert(0, utils_path)

from custom_tokenizer import custom_sent_tokenizer
from text_cleaner import clean_text
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

print(custom_sent_tokenizer("hey are you gonna work? you shouldn't split."))
print(custom_sent_tokenizer("hey are you gonna work? you should split", True))

Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
["hey are you gonna work? you shouldn't split."]
['hey are you gonna work?', 'you should split']


In [3]:



import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence
from sklearn.model_selection import train_test_split








2025-03-18 21:29:05 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2025-03-18 21:29:05 INFO: Downloading default packages for language: en (English) ...
2025-03-18 21:29:07 INFO: File exists: /Users/gbaldonado/stanza_resources/en/default.zip
2025-03-18 21:29:10 INFO: Finished downloading models and saved to /Users/gbaldonado/stanza_resources
2025-03-18 21:29:10 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-18 21:29:10 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2025-03-18 21:29:11 INFO: Loading these models for language: en (English):
| Processor    | Package                   |
--------------------------------------------
| tokenize     | combined                  |
| mwt          | combined                  |
| pos          | combined_charlm           |
| lemma        | combined_nocharlm         |
| constituency | ptb3-revised_charlm       |
| depparse     | combined_charlm           |
| sentiment    | sstplus_charlm            |
| ner          | ontonotes-ww-multi_charlm |

2025-03-18 21:29:11 INFO: Using device: cpu
2025-03-18 21:29:11 INFO: Loading: tokenize
2025-03-18 21:29:11 INFO: Loading: mwt
2025-03-18 21:29:11 INFO: Loading: pos
2025-03-18 21:29:12 INFO: Loading: lemma
2025-03-18 21:29:12 INFO: Loading: constituency
2025-03-18 21:29:12 INFO: Loading: depparse
2025-03-18 21:29:12 INFO: Loading: sentiment
2025-03-18 21:29:12 INFO: Loading: 

## 1.) Load dataset

In [11]:
batch_1_df = pd.read_csv("../../../data/for_training/batch_1_processed/batch_1_dataset_all_capitals_quotes_published_unpublished_IRR.csv") # Needs
batch_2_df = pd.read_excel("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_2_all_themes/batch_2_all_supercapitals_w_nav.xlsx")

In [12]:
# change some col names in batch 2
batch_2_df.rename(columns={"Alma ID": "Alma ID 2", "Essay ID": "Alma ID", "Annotated Essays": "Essay: Why I am here?"}, inplace=True)

**Creating a separate dataset for Navigational**

In [13]:
column_names = ['Alma ID', 
                        'Essay: Why I am here?', 
                        'Attainment', 
                        'First Gen', 
                        'Aspirational', 
                        'Navigational', 
                        'Resistance', 
                        'Perseverance', 
                        'Filial Piety', 
                        'Familial', 
                        'Community Consciousness',
                        'Social', 
                        'Spiritual']

In [14]:
batch_1_df.head()

,Alma ID,Essay: Why I am here?,Attainment,First Gen,Aspirational,Navigational,Resistance,Perseverance,Filial Piety,Familial,Community Consciousness,Social,Spiritual
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,to better one self and be able to succeed late...,0,0,0,0,0,0,0,I always wanted to be able to help people
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0,0,I am also here because the formula to being su...,0,0,0,She made a lot of sacrifices for me to be here...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,0,0,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...,I know that as a child I never thought of edu...,0,I am here because I want to better myself & my...,0,I Hope to reach a position in which I can insp...,0,0
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,0,0,School has always helped me escape the hard th...,"As the oldest of 3 siblings, I want to be a ro...",0,I want to help my community while I'm here,0,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,0,0,- I am here to help myself grow. /%/ - Unders...,Learn and apply my skills to progress towards...,0,0,0,0,0,Meet people who have (similar) aspirations/goa...,0


In [15]:
batch_2_df.head()

,Alma ID 2,Alma ID,Essay: Why I am here?,Aspirational,Familial,Social,Navigational,Resistance
0,542,S20.SCI0115.01.000.542,I am also curious about other Sci classes off...,NaN,NaN,NaN,I am also curious about other Sci classes off...,NaN
1,1349,F20.PHYS0112.04.011.1349,"""Why am I here"", the reason why I am here taki...",0,0,0,"""Why am I here"", the reason why I am here taki...",0
2,376,S20.PHYS0112.04.003.376,"""Why am I here"". I am here because I want to h...",I am here because I want to have more opportu...,0,I believe that humans have intrinsic value and...,the big reason why i chose to come here was al...,0
3,63,F20.PHYS0122.01.012.063,"""Why am I here?"" I saw this question as why am...",I'm here because I want to go to graduate sch...,i'm passionate about the field and i want to u...,I'm passionate about the field and I want to ...,"In order to become a physical therapist, I nee...",0
4,279,S20.ASTR0116.06.021.279,"""Why am I here?"" is a question that runs throu...",am here because I wanted to further my educat...,0,0,I am here because I wanted to further my educ...,0


In [20]:
batch_2_df.isna().sum()

Alma ID 2                0
Alma ID                  0
Essay: Why I am here?    0
Aspirational             0
Familial                 0
Social                   0
Navigational             0
Resistance               0
dtype: int64

In [19]:
batch_2_df = batch_2_df.fillna(0)
batch_2_df

,Alma ID 2,Alma ID,Essay: Why I am here?,Aspirational,Familial,Social,Navigational,Resistance
0,542,S20.SCI0115.01.000.542,I am also curious about other Sci classes off...,0,0,0,I am also curious about other Sci classes off...,0
1,1349,F20.PHYS0112.04.011.1349,"""Why am I here"", the reason why I am here taki...",0,0,0,"""Why am I here"", the reason why I am here taki...",0
2,376,S20.PHYS0112.04.003.376,"""Why am I here"". I am here because I want to h...",I am here because I want to have more opportu...,0,I believe that humans have intrinsic value and...,the big reason why i chose to come here was al...,0
3,63,F20.PHYS0122.01.012.063,"""Why am I here?"" I saw this question as why am...",I'm here because I want to go to graduate sch...,i'm passionate about the field and i want to u...,I'm passionate about the field and I want to ...,"In order to become a physical therapist, I nee...",0
4,279,S20.ASTR0116.06.021.279,"""Why am I here?"" is a question that runs throu...",am here because I wanted to further my educat...,0,0,I am here because I wanted to further my educ...,0
...,...,...,...,...,...,...,...,...
1026,508,S20.PHYS0242.03.006.508,“Why am I here?” Is a weird question to me. Li...,The reason I came to SFSU is to finish my degr...,I’m here because of my family pushing me to do...,0,. I’m in Physics 240 and 242 because it’s a re...,0
1027,1498,F20.PHYS0112.10.019.1498,“Why am I here?” There are multiple reasons as...,one of these goals is being in a career i enjoy.,I am here is because of everything my parents ...,i know that i want a career in which i’ll help...,But I think the main important reason as to w...,0
1028,244,S20.ASTR0116.02.013.244,“Why am I here?” is a very good question to me...,0,0,being in this class is very fun because people...,I would say I need to class to fulfill my GE r...,0
1029,557,S20.SCI0115.02.002.557,“Why am I here?”So why am I here in the class ...,0,0,And notice that the teachers of this class are...,This is my first time taking a tutoring class ...,0


## 2. Create theme datasets

In [21]:
def merge_super_themes(batch_1_df):
    """
    Merges theme annotations into five super themes: Aspirational, Familial Plus, Social Plus, 
    Navigational, and Resistance Plus.

    Parameters:
    - batch_1_df (pd.DataFrame): The original table containing 11 themes.

    Returns:
    - pd.DataFrame: A new DataFrame with merged super themes.
    """

    # Define theme merging rules
    theme_mapping = {
        "Aspirational": ["Attainment", "Aspirational"],
        "Familial": ["Familial", "Filial Piety", "First Gen"],
        "Social": ["Social", "Community Consciousness", "Spiritual"],
        "Navigational": ["Navigational"],
        "Resistance": ["Resistance", "Perseverance"]
    }

    # List to store the new processed rows
    merged_rows = []

    # Iterate through each row
    for _, row in batch_1_df.iterrows():
        merged_row = {
            "Alma ID": row["Alma ID"],
            "Essay: Why I am here?": row["Essay: Why I am here?"]
        }

        # Merge annotations into super themes
        for super_theme, original_themes in theme_mapping.items():
            annotations = []

            for theme in original_themes:
                annotation = str(row[theme]).strip()
                if annotation != "0" and annotation.lower() != "nan":  # Exclude empty and zero values
                    annotations.append(annotation)

            # Merge annotations using our rules
            if len(annotations) > 1:
                # If one annotation fully contains the other, keep the longer version
                longest_annotation = max(annotations, key=len)
                if all(longest_annotation in a or a in longest_annotation for a in annotations):
                    merged_annotation = longest_annotation
                else:
                    # Preserve original order from the essay while merging with "..." if needed
                    merged_annotation = " /%/ ".join(annotations)
            elif len(annotations) == 1:
                merged_annotation = annotations[0]
            else:
                merged_annotation = 0  # No annotation for this super theme

            merged_row[super_theme] = merged_annotation

        # Append the processed row to the list
        merged_rows.append(merged_row)

    # Convert the list into a DataFrame
    merged_df = pd.DataFrame(merged_rows)

    return merged_df

# Example usage:
super_batch_1_df = merge_super_themes(batch_1_df)
super_batch_1_df


,Alma ID,Essay: Why I am here?,Aspirational,Familial,Social,Navigational,Resistance
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...,0,I always wanted to be able to help people,0,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,I am also here because the formula to being su...,I'm here because my parents came here during t...,0,0,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I am here because I want to better myself & my...,I am here because I want to better myself & my...,I Hope to reach a position in which I can insp...,Being in this instituion will pave a way for m...,I know that as a child I never thought of educ...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...,"As the oldest of 3 siblings, I want to be a ro...",I want to help my community while I'm here,0,School has always helped me escape the hard th...
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- I am here to help myself grow. /%/ - Unders...,0,Meet people who have (similar) aspirations/goa...,Learn and apply my skills to progress towards ...,0
...,...,...,...,...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0,My family has valued hard work. Growing up I w...,0,0,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0,my dad told me that it’s going to be hard to g...,0,0,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0,0,The values of my community have made a deep an...,0,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0,Well one thing my mom has taught me is to not ...,0,0,0


In [22]:
theme_names = column_names[2:]
super_theme_names = ["Aspirational", "Familial", "Navigational", "Social", "Resistance"]

def create_theme_datasets(df, themes, sentence_level=True):
    """
    Creates separate datasets for each theme and returns them as a dictionary.

    :param df: The original DataFrame.
    :param themes: List of theme columns to create separate datasets for.
    :return: A dictionary with theme names as keys and DataFrames as values.
    """
    base_columns = ['Alma ID', 'Essay: Why I am here?']
    theme_datasets = {}

    for theme in themes:
        # Select the columns for the theme dataset
        theme_df = df[base_columns + [theme]]
        

        if sentence_level:
        # Filter out rows where the theme value is 0
            theme_df = theme_df[theme_df[theme] != 0]
            theme_df = theme_df[theme_df[theme] != "0"]
        
        # Add the DataFrame to the dictionary
        theme_datasets[theme] = theme_df


    return theme_datasets

# Create the theme datasets
batch_1_by_theme_nonsentencelevel_df = create_theme_datasets(super_batch_1_df, super_theme_names, True)
batch_2_by_theme_nonsentencelevel_df = create_theme_datasets(batch_2_df, super_theme_names, True)

In [23]:
batch_1_by_theme_nonsentencelevel_df["Aspirational"]

,Alma ID,Essay: Why I am here?,Aspirational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
1,S18.W2.111.01.02,I'm here because my parents came here during t...,I am also here because the formula to being su...
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I am here because I want to better myself & my...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- I am here to help myself grow. /%/ - Unders...
...,...,...,...
574,F19.PHYS242.05.03,I am here because my parents want me to finish...,I also want to make a lot of money.
575,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...
576,F19.PHYS242.05.05,"Why am I here? Honestly, I think I'm here by s...",I look forward to furthering my understanding ...
577,F19.PHYS242.05.06,One of the main reasons why I came to Californ...,One of the main reasons why I came to Californ...


**Data Preparation**  
1. Get noun and adjective

In [24]:
def get_noun_adj(phrase):
    noun_adj = []
    for tag in pos_tag(phrase.split()):
        if tag[1] in ['NN','NNP','NNS', 'NNPS','JJ', 'JJR', 'JJS']:
            noun_adj.append(tag[0])
    return  noun_adj

In [25]:
## to replace get_ner??
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

def jaccard_similarity(str1, str2):
    vect = CountVectorizer().fit_transform([str1, str2])
    vectors = vect.toarray()
    return jaccard_score(vectors[0], vectors[1], average='weighted')


3. Get sentence label
    1. Look for exact match
    2. Check for NER in phrase (title/profession/scientific fields) and check if those exist in the sentence
    3. Get the nouns and adjectives and check if those exist in the sentence

In [26]:
def get_sentence_label(essay_sentence, phrase): 
    """
    Determines if an essay sentence should be labeled as positive (1) based on the annotation phrase.
    
    Criteria:
    1. Direct phrase match (regex)
    2. Jaccard similarity score (>0.5 threshold)
    3. Matching nouns & adjectives from the annotation

    Parameters:
    - essay_sentence (str): The essay sentence being checked.
    - phrase (str): The annotation phrase to compare against.

    Returns:
    - int: 1 if the sentence should be labeled positive, otherwise 0.
    """
    
    if not essay_sentence or not phrase:
        return 0  # Return negative if either is empty

    # Step 1: Exact phrase match using regex
    escaped_phrase = re.escape(phrase).replace(r'\/', '/').replace(r'\-', '-')
    if re.search(rf".*{escaped_phrase}.*", essay_sentence, re.IGNORECASE):
        return 1

    # Step 2: Jaccard similarity check
    similarity = jaccard_similarity(essay_sentence, phrase)
    if similarity > 0.5:  # Adjust the threshold as needed
        return 1

    # Step 3: Noun & adjective overlap check
    noun_adj_phrase = get_noun_adj(phrase)
    if all(pos in essay_sentence for pos in noun_adj_phrase):  
        return 1  # If all important words exist, label as positive
    
    return 0  # Default to negative

In [27]:
import os
print(os.getcwd())  # This should be in the "ml-alma-taccti" folder

/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/src/data_preprocessing/prepare_sentence_level


## Looking at example 1, index 0

In [28]:
aspirational_batch_1 = batch_1_by_theme_nonsentencelevel_df["Aspirational"].reset_index(drop=True)
aspirational_batch_1["Essay: Why I am here?"][0]

"Why am I here? Well why does anyone pursue a higher education? to better one self and be able to succeed later on in life. Ever since I was little I wanted to be a doctor, so much so that I can't see myself doing anything else. I always wanted to be able to help people and I always had an intrest in medicine and the human body. I guess I here so I'll be able to help people in the future."

In [29]:
aspirational_batch_1.head(1)

,Alma ID,Essay: Why I am here?,Aspirational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...


## 3. Creating sentence level phrases

In [30]:
def process_sentences_for_theme(theme_df, theme_column):
    """
    Processes sentences for a given theme, tokenizes them, and assigns labels based on annotations.

    Parameters:
    - theme_df (pd.DataFrame): DataFrame containing essays and annotations.
    - theme_column (str): The column corresponding to the theme annotation.

    Returns:
    - pd.DataFrame: A DataFrame containing processed sentences, their labels, and phrases.
    """
    processed_rows = []  # Use a list for efficient row collection

    for _, row in theme_df.iterrows():
        # Extract essay and annotation
        text = row['Essay: Why I am here?'].strip()
        annotation = str(row[theme_column]).strip()

        # Tokenize essay into sentences
        essay_sentences = [clean_text(x).lstrip() for x in custom_sent_tokenizer(text, True) if x.strip()]
        
        # Split annotation into individual phrases using "/%/"
        annotation_phrases = [clean_text(x).lstrip() for x in custom_sent_tokenizer(annotation) if x.strip()]

        # Annotate each sentence
        for sentence in essay_sentences:
            if not re.search(r'\w', sentence) or len(sentence.split()) < 3:
                continue  # Skip empty or very short sentences
            
            # Default label is 0
            label = 0
            for phrase in annotation_phrases:
                if len(phrase.split()) > 2:
                    label = get_sentence_label(sentence, phrase)
                    if label == 1:
                        break  # Stop checking once a match is found

            # Append to list instead of using pd.concat (better performance)
            processed_rows.append({'sentence': sentence.lower(), 'label': label, 'phrase': annotation_phrases})

    # Convert collected rows into a DataFrame
    return pd.DataFrame(processed_rows, columns=['sentence', 'label', 'phrase'])

In [31]:
from custom_tokenizer import custom_sent_tokenizer
from text_cleaner import clean_text

## EXAMPLE #QUALITY CONTROL
# Extract text and annotation
text = batch_1_by_theme_nonsentencelevel_df["Aspirational"]["Essay: Why I am here?"][11]
annotation = batch_1_by_theme_nonsentencelevel_df["Aspirational"]["Aspirational"][11]

print(text)
print(annotation)
print(custom_sent_tokenizer(annotation))

# Tokenize essay and annotation
essay_sentences = [clean_text(x) for x in custom_sent_tokenizer(text) if x.strip()]
annotation_sentences = [clean_text(x) for x in custom_sent_tokenizer(annotation) if x.strip()]


print("Annotation strip:", annotation_sentences)
print("Essay strip:", essay_sentences)

# Annotate the essay on a sentence level
for sentence in essay_sentences:
    if not re.search(r'\w', sentence) or len(sentence.split()) < 3:
        continue  # Skip empty or very short sentences
    label = 0
    for annotation_phrase in annotation_sentences:
        if len(annotation_phrase.split()) > 2:
            label = get_sentence_label(sentence, annotation_phrase)
            if label == 1:
                break  # Stop checking once a match is found

I am here at SFSU for a better future. A future where I don't have to struggle at the end of the month to pay bills or provide for my family. To achieve goals such as open up a homeless shelter & pay for a house for my mother so that she doesn't have to work another day in her life. I aspire to become a doctor because I know that not only can I change my life, families, but also the people around me. As for why I am here in San Francisco, I moved away to isolate myself from any distractions that would cause me to not reach my highest potential.
I aspire to become a doctor /%/  To achieve goals such as open up a homeless shelter /%/ I am here at SFSU for a better future /%/ I'm taking this class to get a good grade in physics and to understand the material.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
['I aspire to become a doctor', 'To achieve goals such as open up a homeless shelter', 'I am here at SFSU for a 

In [32]:
batch_1_by_theme_nonsentencelevel_df["Aspirational"]

,Alma ID,Essay: Why I am here?,Aspirational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
1,S18.W2.111.01.02,I'm here because my parents came here during t...,I am also here because the formula to being su...
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I am here because I want to better myself & my...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,I want to help my community while I'm here and...
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- I am here to help myself grow. /%/ - Unders...
...,...,...,...
574,F19.PHYS242.05.03,I am here because my parents want me to finish...,I also want to make a lot of money.
575,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...
576,F19.PHYS242.05.05,"Why am I here? Honestly, I think I'm here by s...",I look forward to furthering my understanding ...
577,F19.PHYS242.05.06,One of the main reasons why I came to Californ...,One of the main reasons why I came to Californ...


In [33]:
# Initialize dictionaries to store processed datasets for Batch 1 and Batch 2
batch_1_sentence_level_datasets = {}
batch_2_sentence_level_datasets = {}

# Loop through each super theme and process sentences
for theme in super_theme_names:
    batch_1_sentence_level_datasets[theme] = process_sentences_for_theme(batch_1_by_theme_nonsentencelevel_df[theme], theme)
    batch_2_sentence_level_datasets[theme] = process_sentences_for_theme(batch_2_by_theme_nonsentencelevel_df[theme], theme)

# Display results
print("✅ Batch 1 and Batch 2 sentence-level datasets processed successfully!")

Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a de

In [35]:
batch_1_sentence_level_datasets["Aspirational"]

,sentence,label,phrase
0,why am i here?,0,[ever since i was little i wanted to be a doct...
1,well why does anyone pursue a higher education?,0,[ever since i was little i wanted to be a doct...
2,to better one self and be able to succeed late...,1,[ever since i was little i wanted to be a doct...
3,ever since i was little i wanted to be a docto...,1,[ever since i was little i wanted to be a doct...
4,i always wanted to be able to help people and ...,1,[ever since i was little i wanted to be a doct...
...,...,...,...
3492,"i want to better myself, improve myself in eve...",1,[i want to be able to apply my knowledge to my...
3493,i have no problems applying myself when it com...,0,[i want to be able to apply my knowledge to my...
3494,grow mentally so that i can apply more physica...,0,[i want to be able to apply my knowledge to my...
3495,use what i learn and apply it in whatever care...,0,[i want to be able to apply my knowledge to my...


In [ ]:
# QUALITY CHECK: GOOD
batch_1_sentence_level_datasets.to_excel("batch_1_aspirational_plus.xlsx") 

In [26]:
batch_2_sentence_level_datasets = process_sentences_for_theme(batch_2_by_theme_nonsentencelevel_df["Aspirational"], "Aspirational")
batch_2_sentence_level_datasets

Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a delimiter because it often appears in rhetorical questions or as part of the essay prompt.
Note: (?) is NOT treated as a de

,sentence,label,phrase
0,"""why am i here"".",0,[i am here because i want to have more opportu...
1,i am here because i want to have more opportun...,1,[i am here because i want to have more opportu...
2,college is feel is a big stepping stone for yo...,0,[i am here because i want to have more opportu...
3,"but more specifically, i came to sf state to f...",0,[i am here because i want to have more opportu...
4,the big reason why i chose to come here was al...,0,[i am here because i want to have more opportu...
...,...,...,...
5218,"anyways, the path im referring to is basically...",0,[one of these goals is being in a career i enj...
5219,i know that i want a career in which ill help ...,0,[one of these goals is being in a career i enj...
5220,"in any case, i intend to keep moving forward i...",0,[one of these goals is being in a career i enj...
5221,i am here to fulfill a covenant with myself an...,1,[i am here to fulfill a covenant with myself a...


In [27]:
batch_2_sentence_level_datasets

,sentence,label,phrase
0,"""why am i here"".",0,[i am here because i want to have more opportu...
1,i am here because i want to have more opportun...,1,[i am here because i want to have more opportu...
2,college is feel is a big stepping stone for yo...,0,[i am here because i want to have more opportu...
3,"but more specifically, i came to sf state to f...",0,[i am here because i want to have more opportu...
4,the big reason why i chose to come here was al...,0,[i am here because i want to have more opportu...
...,...,...,...
5218,"anyways, the path im referring to is basically...",0,[one of these goals is being in a career i enj...
5219,i know that i want a career in which ill help ...,0,[one of these goals is being in a career i enj...
5220,"in any case, i intend to keep moving forward i...",0,[one of these goals is being in a career i enj...
5221,i am here to fulfill a covenant with myself an...,1,[i am here to fulfill a covenant with myself a...


In [28]:
# QUALITY CHECK: GOOD
batch_2_sentence_level_datasets.to_excel("batch_2_aspirational_plus.xlsx") 

In [29]:
aspirational_plus_batch_1_batch_2_merged = pd.concat([batch_1_sentence_level_datasets, batch_2_sentence_level_datasets])
aspirational_plus_batch_1_batch_2_merged

,sentence,label,phrase
0,why am i here?,0,[ever since i was little i wanted to be a doct...
1,well why does anyone pursue a higher education?,0,[ever since i was little i wanted to be a doct...
2,to better one self and be able to succeed late...,1,[ever since i was little i wanted to be a doct...
3,ever since i was little i wanted to be a docto...,1,[ever since i was little i wanted to be a doct...
4,i always wanted to be able to help people and ...,1,[ever since i was little i wanted to be a doct...
...,...,...,...
5218,"anyways, the path im referring to is basically...",0,[one of these goals is being in a career i enj...
5219,i know that i want a career in which ill help ...,0,[one of these goals is being in a career i enj...
5220,"in any case, i intend to keep moving forward i...",0,[one of these goals is being in a career i enj...
5221,i am here to fulfill a covenant with myself an...,1,[i am here to fulfill a covenant with myself a...


In [31]:
aspirational_plus_batch_1_batch_2_merged.to_excel("aspirational_plus_batch_1_batch_2_merged.xlsx", index=False)

In [36]:
import os

# Define save path
save_dir = "/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/"
os.makedirs(save_dir, exist_ok=True)  # Ensure the directory exists

# Loop through each theme, save batch files, and merge them
for theme in super_theme_names:
    # Define file names
    batch_1_filename = f"{save_dir}batch_1/batch_1_{theme}_plus.xlsx"
    batch_2_filename = f"{save_dir}batch_2/batch_2_{theme}_plus.xlsx"
    merged_filename = f"{save_dir}merged/{theme}_plus_batch_1_batch_2_merged.xlsx"

    # Save Batch 1 and Batch 2 datasets
    batch_1_sentence_level_datasets[theme].to_excel(batch_1_filename, index=False)
    batch_2_sentence_level_datasets[theme].to_excel(batch_2_filename, index=False)

    # Merge Batch 1 and Batch 2, then save
    merged_df = pd.concat([batch_1_sentence_level_datasets[theme], batch_2_sentence_level_datasets[theme]], ignore_index=True)
    merged_df.to_excel(merged_filename, index=False)

    print(f"✅ Saved: {batch_1_filename}, {batch_2_filename}, {merged_filename}")

print("✅ All Batch 1, Batch 2, and merged theme datasets have been successfully saved!")

✅ Saved: /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1/batch_1_Aspirational_plus.xlsx, /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_2/batch_2_Aspirational_plus.xlsx, /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/Aspirational_plus_batch_1_batch_2_merged.xlsx
✅ Saved: /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1/batch_1_Familial_plus.xlsx, /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_2/batch_2_Familial_plus.xlsx, /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/merged/Familial_plus_batch_1_batch_2_merged.xlsx
✅ Saved: /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_1/batch_1_Navigational_plus.xlsx, /Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/for_training/batch_2/batch_2_Navigational_plus.xlsx, /Users/gbaldonado/Developer/ml-

In [211]:
def process_essays_for_theme(theme_df, theme_column):
    theme_essay_df = pd.DataFrame(columns=['essay', 'label', 'phrase'])

    essay_from_df = theme_df["Essay: Why I am here?"]
    essay_list = [clean_text(essay) for essay in essay_from_df]

    label_list = [1 if value != "0" and value != 0 else 0 for value in theme_df[theme_column]]
    annotation_from_df = theme_df[theme_column]
    annotation_list = []
    
    for annotation in annotation_from_df:
        annotation_instances = re.split(r'/%/', annotation)
        annotation_instances = [clean_text(a) for a in annotation_instances]
        annotation_list.append(annotation_instances)

    theme_essay_df["essay"] = essay_list
    theme_essay_df["label"] = label_list
    theme_essay_df["phrase"] = annotation_list

    return theme_essay_df

In [212]:
processed_essay_level_datasets = process_essays_for_theme(essay_level_theme_datasets["Navigational"], "Navigational")
processed_essay_level_datasets

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,1,[Being in this instituion will pave a way for ...
3,I am here at SFSU because I am a first generat...,0,[0]
4,I am here to help myself grow. Learn and apply...,1,[Learn and apply my skills to progress towards...
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [32]:
essay_level_theme_names = sentence_level_theme_names
essay_level_theme_names

NameError: name 'sentence_level_theme_names' is not defined

In [218]:
processed_essay_theme_datasets = {}

for i in sentence_level_theme_names:
    print("Working on " + i)
    processed_name = f"{i}_processed_essay_df"
    processed_essay_theme_datasets[i] = process_essays_for_theme(essay_level_theme_datasets[i], i)

Working on Navigational
Working on Perseverance
Working on Resistance


In [220]:
processed_essay_theme_datasets["Perseverance"]

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,0,[0]
3,I am here at SFSU because I am a first generat...,1,[School has always helped me escape the hard t...
4,I am here to help myself grow. Learn and apply...,0,[0]
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [213]:
essay_level_theme_datasets["Navigational"]

,Alma ID,Essay: Why I am here?,Navigational
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,0
1,S18.W2.111.01.02,I'm here because my parents came here during t...,0
2,S18.W2.111.01.03,I am here because I want to better myself & my...,Being in this instituion will pave a way for m...
3,S18.W2.111.01.04,I am here at SFSU because I am a first generat...,0
4,S18.W2.111.01.05,I am here to help myself grow.\n- Learn and a...,- Learn and apply my skills to progress toward...
...,...,...,...
584,F19.PHYS242.02.004.1216,My family has valued hard work. Growing up I w...,0
585,F19.PHYS242.02.005.1213,I have lived in the Bay Area my whole life and...,0
586,F19.PHYS242.02.006.1217,The values of my community have made a deep an...,0
587,F19.PHYS242.02.007.1211,Well one thing my mom has taught me is to not ...,0


In [83]:
sentence_level_theme_names = ["Navigational", "Perseverance", "Resistance"]

In [96]:
sentence_level_theme_names = ["Attainment", "Aspirational", "Familial", "Filial Piety", "First Gen", "Community Consciousness", "Spiritual", "Social"]

In [85]:
sentence_level_theme_names

['Navigational', 'Perseverance', 'Resistance']

In [30]:
sentence_level_theme_datasets

{'Attainment':                Alma ID                              Essay: Why I am here?  \
 0     S18.W2.111.01.01  Why am I here? Well why does anyone pursue a h...   
 6     S18.W2.111.01.07  I am here to better myself and hopefully give ...   
 8     S18.W2.111.01.09  I am here in this class because I want a littl...   
 11    S18.W2.111.01.12  I am here at SFSU for a better future. A futur...   
 16    S18.W2.111.01.17  I'm here because I want to improve my understa...   
 ..                 ...                                                ...   
 568  F19.PHYS102.02.08  Why am I here? If you're asking me why am I on...   
 572  F19.PHYS242.05.01  I am here at SFSU for a multitude of reasons. ...   
 573  F19.PHYS242.05.02  I am here because I want to make my mom proud....   
 575  F19.PHYS242.05.04  Why am I here? †I am at San Francisco State Un...   
 577  F19.PHYS242.05.06  One of the main reasons why I came to Californ...   
 
                                             Att

In [162]:
navigational_processed_sentence_df_2 = process_sentences_for_theme(sentence_level_theme_datasets["Resistance"], "Resistance")
navigational_processed_sentence_df_2

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[I know that as a child I never thought of edu...
1,being in this instituion will pave a way for m...,0,[I know that as a child I never thought of edu...
2,i know that as a child i never thought of educ...,1,[I know that as a child I never thought of edu...
3,i hope to reach a position in which i can insp...,1,[I know that as a child I never thought of edu...
4,"also, statistics have shown that people with h...",0,[I know that as a child I never thought of edu...
...,...,...,...
152,im currently lost as to how it is that i want ...,0,"[I want to represent myself, my gender, my sex..."
153,i am here because of the way that i have grown...,0,[I am here to disprove stereotypes.]
154,im here because of all the sacrifices that bot...,0,[I am here to disprove stereotypes.]
155,im here to disapprove stereotypes im here for ...,0,[I am here to disprove stereotypes.]


In [120]:
navigational_processed_sentence_df_2[navigational_processed_sentence_df_2["label"] == 1]

16

In [224]:
attainment_processed_sentence_df["label"].value_counts()

label
0    1437
1     304
Name: count, dtype: int64

In [223]:
attainment_processed_sentence_df_2["label"].value_counts()

label
0    1407
1     261
Name: count, dtype: int64

In [221]:
attainment_processed_sentence_df_2
attainment_processed_sentence_df_2.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_sentence_level_batch_1_jaccard.csv", index=False)

In [91]:
sentence_level_theme_names

['Navigational', 'Perseverance', 'Resistance']

NameError: name 'processed_theme_datasets' is not defined

In [31]:
sentence_level_theme_datasets["Resistance"]

,Alma ID,Essay: Why I am here?,Resistance
2,S18.W2.111.01.03,I am here because I want to better myself & my...,I know that as a child I never thought of edu...
20,S18.W2.111.01.22,I am here because I care about my community an...,I am here in the science community because I f...
30,F18.SCI111.03.09,I am here because I am dedicated to my succes...,I’m here for all the little colored girls who ...
52,F18.SI.111.04.07,I am here because I am a first generation coll...,I want to show all the minorities that you can...
69,S19.SCI111.01.10,I am here because I always wanted to be involv...,"I am an African American male, and there is n..."
70,S19.SCI111.02.01,I am here in this world to make an impact. I a...,Not only am I making an impact on the medical ...
182,F19.PHYS122.01.02,I am here to take this physics class so I can ...,Iíve seen what good and bad doctors have done ...
190,F19.PHYS122.01.12,I am here for a list of reasons. Reasons that ...,"Firstly, I am here to challenge myself and mak..."
192,F19.PHYS122.01.14,Why I am here is because I want to fulfill my ...,I was born in Afghanistan in a house during th...
214,F19.PHYS122.08.06,Iím in college because I know that I can make ...,I have a drive to do better in a world that co...


In [ ]:
sentence_level_theme_names = ["Aspirational", ""Navigational", "Perseverance", "Resistance"]

In [34]:
processed_theme_datasets = {}

for i in sentence_level_theme_names: #skipping Attainment because we already did it
    print("Working on " + i)
    processed_name = f"{i}_processed_sentence_df"
    processed_theme_datasets[i] = process_sentences_for_theme(sentence_level_theme_datasets[i], i)

Working on Navigational
Working on Perseverance
Working on Resistance


In [96]:
sentence_level_theme_datasets["Navigational"]["Essay: Why I am here?"][565]

"I am here to learn the basics of physics through experiments, also to graduate as the first one in my family from a college university. This should help me in the future because it is for my major requirement for kinesiology. I am also enrolled in the physics lecture. Physics is in everyday life so this class will come in handy. Currently I'm learning conversions which I am bad at. I wish to improve my math skills and conversion skills. This class will prepare me for upper division classes, even though its my last physics class, it should come in handy in everyday problem solving. For example if I ever need to calculate how much gas I need to pump in my car. I can calculate it with money, gallons, and miles. So this class will enhance my problem solving skills right now in life, and later in life. I am here to learn.†"

In [299]:
from utils.text_cleaner import clean_text



In [35]:
for k,v in processed_theme_datasets.items():
    print(k, len(v))

Navigational 3196
Perseverance 1047
Resistance 157


: 

In [170]:
processed_theme_datasets["Navigational"]

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[Being in this instituion will pave a way for ...
1,being in this instituion will pave a way for m...,1,[Being in this instituion will pave a way for ...
2,i know that as a child i never thought of educ...,0,[Being in this instituion will pave a way for ...
3,i hope to reach a position in which i can insp...,0,[Being in this instituion will pave a way for ...
4,"also, statistics have shown that people with h...",0,[Being in this instituion will pave a way for ...
...,...,...,...
3191,"i want to better myself, improve myself in eve...",0,[I am here to fulfill my mechanical engineerin...
3192,i have no problems applying myself when it com...,0,[I am here to fulfill my mechanical engineerin...
3193,grow mentally so that i can apply more physica...,0,[I am here to fulfill my mechanical engineerin...
3194,use what i learn and apply it in whatever care...,0,[I am here to fulfill my mechanical engineerin...


In [230]:
aspirational_processed_sentence_df = processed_theme_datasets["Aspirational"]
aspirational_processed_sentence_df.to_csv("")

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,1,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [225]:
for k,v in processed_essay_theme_datasets.items():
    print(k, len(v))
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_essay_level/{k}_essay_level_batch_1_jaccard.csv", index=False)

Navigational 589
Perseverance 589
Resistance 589


In [172]:
for k,v in processed_theme_datasets.items():
    print(k, len(v))
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/single_theme_using_jaccard_method/{k}_sentence_level_batch_1_jaccard.csv", index=False)

Navigational 3196
Perseverance 1047
Resistance 157


In [240]:
merged_resistance_essay_df = pd.concat([processed_essay_theme_datasets["Perseverance"], processed_essay_theme_datasets["Resistance"]])
merged_resistance_essay_df.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/essay_level/merged_themes_essay_level/merged_Resistance_essay_level_batch_1.csv", index=False)

In [243]:
pd.read_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/essay_level/merged_themes_essay_level/merged_Resistance_essay_level_batch_1.csv")


,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,['0']
1,I'm here because my parents came here during t...,0,['0']
2,I am here because I want to better myself my f...,0,['0']
3,I am here at SFSU because I am a first generat...,1,['School has always helped me escape the hard ...
4,I am here to help myself grow. Learn and apply...,0,['0']
...,...,...,...
1173,My family has valued hard work. Growing up I w...,0,['0']
1174,I have lived in the Bay Area my whole life and...,0,['0']
1175,The values of my community have made a deep an...,0,['0']
1176,Well one thing my mom has taught me is to not ...,0,['0']


## 3. Merging similar themes 

1. Attainment and aspirational
2. Familial, filial piety, and first-generation
3. Perserverance and resistance (essay level)
4. Community consciousness, spiritual, and social
5. Navigational (essay level)


In [176]:
processed_theme_datasets["Navigational"]

,sentence,label,phrase
0,i am here because i want to better myself my f...,0,[Being in this instituion will pave a way for ...
1,being in this instituion will pave a way for m...,1,[Being in this instituion will pave a way for ...
2,i know that as a child i never thought of educ...,0,[Being in this instituion will pave a way for ...
3,i hope to reach a position in which i can insp...,0,[Being in this instituion will pave a way for ...
4,"also, statistics have shown that people with h...",0,[Being in this instituion will pave a way for ...
...,...,...,...
3191,"i want to better myself, improve myself in eve...",0,[I am here to fulfill my mechanical engineerin...
3192,i have no problems applying myself when it com...,0,[I am here to fulfill my mechanical engineerin...
3193,grow mentally so that i can apply more physica...,0,[I am here to fulfill my mechanical engineerin...
3194,use what i learn and apply it in whatever care...,0,[I am here to fulfill my mechanical engineerin...


In [323]:
[processed_theme_datasets[i] for i in merged_themes["Aspirational"]]

,sentence,label,phrase
0,why am i here?,0,[to better one self and be able to succeed lat...
1,well why does anyone pursue a higher education?,0,[to better one self and be able to succeed lat...
2,to better one self and be able to succeed late...,1,[to better one self and be able to succeed lat...
3,ever since i was little i wanted to be a docto...,0,[to better one self and be able to succeed lat...
4,i always wanted to be able to help people and ...,0,[to better one self and be able to succeed lat...
...,...,...,...
3031,"i want to better myself, improve myself in eve...",0,[I want to be able to apply my knowledge to my...
3032,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3033,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3034,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


In [229]:
merged_resistance_essay_df

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,0,[0]
3,I am here at SFSU because I am a first generat...,1,[School has always helped me escape the hard t...
4,I am here to help myself grow. Learn and apply...,0,[0]
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [173]:


# merged_themes = {"Aspirational": ["Attainment", "Aspirational"],
#                  "Familial": ["Familial", "Filial Piety", "First Gen"],
#                  "Social": ["Community Consciousness", "Spiritual", "Social"]}


# merged_aspirational_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Aspirational"]])
# merged_familial_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Familial"]])
# merged_social_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Social"]])

merged_themes = {"Resistance": ["Resistance", "Perseverance"]}
merged_resistance_df = pd.concat([processed_theme_datasets[i] for i in merged_themes["Resistance"]])

In [328]:
len(attainment_processed_sentence_df[attainment_processed_sentence_df["label"] == 1])

304

In [247]:
processed_essay_theme_datasets["Navigational"]

,essay,label,phrase
0,Why am I here? Well why does anyone pursue a h...,0,[0]
1,I'm here because my parents came here during t...,0,[0]
2,I am here because I want to better myself my f...,1,[Being in this instituion will pave a way for ...
3,I am here at SFSU because I am a first generat...,0,[0]
4,I am here to help myself grow. Learn and apply...,1,[Learn and apply my skills to progress towards...
...,...,...,...
584,My family has valued hard work. Growing up I w...,0,[0]
585,I have lived in the Bay Area my whole life and...,0,[0]
586,The values of my community have made a deep an...,0,[0]
587,Well one thing my mom has taught me is to not ...,0,[0]


In [249]:
merged_essay_level_datasets = {"Resistance": merged_resistance_essay_df,
"Navigational": processed_essay_theme_datasets["Navigational"]
                                  }

for k, v in merged_essay_level_datasets.items():
    print(k, v["label"].value_counts(), f"percentage of themed essays: {round(len(v[v['label'] == 1]) / len(v) * 100, 2)}%")


Resistance label
0    1031
1     147
Name: count, dtype: int64 percentage of themed essays: 12.48%
Navigational label
1    401
0    188
Name: count, dtype: int64 percentage of themed essays: 68.08%


In [177]:
merged_sentence_level_datasets = {"Resistance": merged_resistance_df,
"Navigational": processed_theme_datasets["Navigational"]
                                  }

for k, v in merged_sentence_level_datasets.items():
    print(k, v["label"].value_counts(), f"percentage of themed essays: {round(len(v[v['label'] == 1]) / len(v) * 100, 2)}%")


Resistance label
0    1117
1      87
Name: count, dtype: int64 percentage of themed essays: 7.23%
Navigational label
0    2912
1     284
Name: count, dtype: int64 percentage of themed essays: 8.89%


In [178]:
for k,v in merged_sentence_level_datasets.items():
    v.to_csv(f"/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/merged_themes_using_jaccard_method/merged_{k}_sentence_level_batch_1_jaccard.csv", index=False)

In [118]:
processed_theme_datasets.keys()

dict_keys(['Aspirational_processed_sentence_df', 'Familial_processed_sentence_df', 'Filial Piety_processed_sentence_df', 'First Gen_processed_sentence_df', 'Community Consciousness_processed_sentence_df', 'Spiritual_processed_sentence_df', 'Social_processed_sentence_df'])

In [119]:
atn_asp_df = pd.concat([attainment_processed_sentence_df, aspirational_processed_sentence_df])
atn_asp_df

,sentence,label,phrase
0,why am i here,0,[Ever since I was little I wanted to be a doct...
1,well why does anyone pursue a higher education,0,[Ever since I was little I wanted to be a doct...
2,to better one self and be able to succeed late...,0,[Ever since I was little I wanted to be a doct...
3,ever since i was little i wanted to be a docto...,1,[Ever since I was little I wanted to be a doct...
4,i always wanted to be able to help people and ...,0,[Ever since I was little I wanted to be a doct...
...,...,...,...
3151,"i want to better myself, improve myself in eve...",1,[I want to be able to apply my knowledge to my...
3152,i have no problems applying myself when it com...,0,[I want to be able to apply my knowledge to my...
3153,grow mentally so that i can apply more physica...,0,[I want to be able to apply my knowledge to my...
3154,use what i learn and apply it in whatever care...,0,[I want to be able to apply my knowledge to my...


/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti


In [127]:
atn_asp_df.to_csv("/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/atn_asp_sentence_level_batch_1.csv", index=False)


In [56]:
attainment_sentence_df = sentence_level_theme_datasets["Attainment"]
attainment_sentence_df

,Alma ID,Essay: Why I am here?,Attainment
0,S18.W2.111.01.01,Why am I here? Well why does anyone pursue a h...,Ever since I was little I wanted to be a docto...
6,S18.W2.111.01.07,I am here to better myself and hopefully give ...,I am in this class because I want to pass phys...
8,S18.W2.111.01.09,I am here in this class because I want a littl...,I am majoring in bio to get to my career goal ...
11,S18.W2.111.01.12,I am here at SFSU for a better future. A futur...,I aspire to become a doctor /%/ To achieve goa...
16,S18.W2.111.01.17,I'm here because I want to improve my understa...,"In the end, the goal is to achieve an A for my..."
...,...,...,...
68,F19.PHYS102.02.08,Why am I here? If you're asking me why am I on...,I would tell you that I am chasing a degree. I...
72,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...
73,F19.PHYS242.05.02,I am here because I want to make my mom proud....,I want to graduate
75,F19.PHYS242.05.04,Why am I here? †I am at San Francisco State Un...,I am at San Francisco State University to get ...


In [78]:
attainment_processed_sentence_df = process_sentences_for_theme(attainment_sentence_df, "Attainment")

KeyboardInterrupt: 

In [94]:
attainment_processed_sentence_df[14:]

,sentence,label,phrase
14,i knew this step was going to be an important ...,0,[I am majoring in bio to get to my career goal...
15,i am here at sfsu for a better future,0,"[I aspire to become a doctor , To achieve goal..."
16,a future where i don't have to struggle at the...,0,"[I aspire to become a doctor , To achieve goal..."
17,to achieve goals such as open up a homeless sh...,1,"[I aspire to become a doctor , To achieve goal..."
18,i aspire to become a doctor because i know tha...,1,"[I aspire to become a doctor , To achieve goal..."
...,...,...,...
1736,†by the time i got here at sfsu i decided to j...,0,[I am at San Francisco State University to get...
1737,†right now i am slightly considering switching...,0,[I am at San Francisco State University to get...
1738,one of the main reasons why i came to californ...,1,[One of the main reasons why I came to Califor...
1739,i move to san francisco because i got accepted...,0,[One of the main reasons why I came to Califor...


In [93]:
attainment_processed_sentence_df["sentence"][18]

'i aspire to become a doctor because i know that not only can i change my life, families, but also the people around me'

**Generate csv file for Attainment**

In [131]:
attainment_df = theme_datasets["Attainment"]

In [132]:
attainment_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('/Users/gbaldonado/Developer/ml-alma-taccti/ml-alma-taccti/data/processed_for_model/using_original_method/attainment_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in attainment_df.iterrows():
        text = val['Essay: Why I am here?'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['Attainment']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])

            # Concatenate the new row to the existing DataFrame
            attainment_sentence_df = pd.concat([attainment_sentence_df, new_row], ignore_index=True)
            # attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)

In [59]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [58]:
attainment_sentence_df

,sentence,label,phrase


In [27]:
pos_labels = len([n for n in attainment_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(attainment_sentence_df['label']), (pos_labels/len(attainment_sentence_df['label']))*100))


Positive labels present in the dataset : 253  out of 1413 or 17.905166312809627%


In [28]:
train, test = train_test_split(attainment_sentence_df, test_size=0.1, random_state=18, stratify=attainment_sentence_df['label'])

In [29]:
train.to_csv("data/csv/attainment_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [30]:
test.to_csv("data/csv/attainment_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [31]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 17.94% positive class
positive labels : 228 out of total 1271 datapoints


In [32]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 17.61% positive class
positive labels : 25 out of total 142 datapoints


**Generate csv file for First Generation**

In [33]:
first_generation_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('data/csv/first_generation_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in first_generation_df.iterrows():
        text = val['Essay : Why am I here'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['First Gen - FINAL']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            

In [34]:
first_generation_sentence_df.shape

(128, 3)

In [35]:
pos_labels = len([n for n in first_generation_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(first_generation_sentence_df['label']), (pos_labels/len(first_generation_sentence_df['label']))*100))


Positive labels present in the dataset : 19  out of 128 or 14.84375%


In [36]:
train, test = train_test_split(first_generation_sentence_df, test_size=0.2, random_state=18, stratify=first_generation_sentence_df['label'])

In [37]:
train.to_csv("data/csv/first_generation_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [38]:
test.to_csv("data/csv/first_generation_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [39]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 14.71% positive class
positive labels : 15 out of total 102 datapoints


In [40]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 15.38% positive class
positive labels : 4 out of total 26 datapoints
